# Разработка модели предсказания потери почтовых отправлений

In [15]:
import pandas as pd
import seaborn as sns

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

from sklearn.metrics import f1_score

c:\Users\Dmitry.Khodykin\Documents\dev\envs\lamlvenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset

In [2]:
train_df = pd.read_csv("Data/train_dataset_train.csv")
train_df.head(3)

C:\Users\Dmitry.Khodykin\AppData\Local\Temp\ipykernel_6880\1356982212.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("Data/train_dataset_train.csv")


,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629.0,Участок,7503.0,N,0.0,Y,N,87.0,...,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976.0,ММПО,7503.0,N,0.0,N,N,107.0,...,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962.0,Цех,7503.0,N,0.0,Y,N,50.0,...,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0


In [13]:
train_df.info(null_counts=True)

C:\Users\Dmitry.Khodykin\AppData\Local\Temp\ipykernel_6880\602459805.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  train_df.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000000 entries, 0 to 5999999
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   id                             6000000 non-null  int64  
 1   oper_type + oper_attr          6000000 non-null  object 
 2   index_oper                     6000000 non-null  object 
 3   type                           6000000 non-null  object 
 4   priority                       6000000 non-null  float64
 5   is_privatecategory             6000000 non-null  object 
 6   class                          6000000 non-null  float64
 7   is_in_yandex                   6000000 non-null  object 
 8   is_return                      6000000 non-null  object 
 9   weight                         6000000 non-null  float64
 10  mailtype                       6000000 non-null  float64
 11  mailctg                        6000000 non-null  float64
 12  mailrank      

In [10]:
train_df["label"].value_counts()

0    5829020
1     170980
Name: label, dtype: int64

In [16]:
test_df = pd.read_csv("Data/test_dataset_test.csv")
test_df.head(3)

C:\Users\Dmitry.Khodykin\AppData\Local\Temp\ipykernel_6880\2616223715.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("Data/test_dataset_test.csv")


,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address
0,7815282,8_13,102976.0,ММПО,7503.0,N,0.0,N,N,85.0,...,489.0,972.0,68766046.0,91123247.0,159889293.0,164927295.0,0,0,0,0
1,8443555,8_2,238753.0,ГОПС,7503.0,N,4.0,Y,N,21.0,...,186.0,2.0,2895.0,1545.0,4440.0,20623.0,0,1,1,0
2,6352559,1020_-1,618254.0,ГОПС,7503.0,N,4.0,Y,N,388.0,...,500.0,3.0,2751.0,993.0,3744.0,37817.0,0,0,0,0


In [17]:
test_df.info(null_counts=True)

C:\Users\Dmitry.Khodykin\AppData\Local\Temp\ipykernel_6880\198631561.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  test_df.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000000 entries, 0 to 3999999
Data columns (total 28 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   id                             4000000 non-null  int64  
 1   oper_type + oper_attr          4000000 non-null  object 
 2   index_oper                     4000000 non-null  object 
 3   type                           4000000 non-null  object 
 4   priority                       4000000 non-null  float64
 5   is_privatecategory             4000000 non-null  object 
 6   class                          4000000 non-null  float64
 7   is_in_yandex                   4000000 non-null  object 
 8   is_return                      4000000 non-null  object 
 9   weight                         4000000 non-null  float64
 10  mailtype                       4000000 non-null  float64
 11  mailctg                        4000000 non-null  float64
 12  mailrank      

## FE

In [20]:
train_df.columns

Index(['id', 'oper_type + oper_attr', 'index_oper', 'type', 'priority',
       'is_privatecategory', 'class', 'is_in_yandex', 'is_return', 'weight',
       'mailtype', 'mailctg', 'mailrank', 'directctg', 'transport_pay',
       'postmark', 'name_mfi', 'weight_mfi', 'price_mfi',
       'dist_qty_oper_login_1', 'total_qty_oper_login_1',
       'total_qty_oper_login_0', 'total_qty_over_index_and_type',
       'total_qty_over_index', 'is_wrong_sndr_name', 'is_wrong_rcpn_name',
       'is_wrong_phone_number', 'is_wrong_address', 'label'],
      dtype='object')

## Fit Model

In [18]:
automl = TabularAutoML(
    task = Task(
        name = "binary"
    )
)

In [50]:
oof_pred = automl.fit_predict(
    train_df,
    roles = {"target": "label", "drop": ["id", "index_oper"]}
)

c:\Users\Dmitry.Khodykin\Documents\dev\envs\lamlvenv\lib\site-packages\lightautoml\transformers\categorical.py:1029: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cnts = cnts.append(Series([cnts.shape[0] + 1], index=[np.nan]))
c:\Users\Dmitry.Khodykin\Documents\dev\envs\lamlvenv\lib\site-packages\lightautoml\transformers\categorical.py:1029: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cnts = cnts.append(Series([cnts.shape[0] + 1], index=[np.nan]))
c:\Users\Dmitry.Khodykin\Documents\dev\envs\lamlvenv\lib\site-packages\lightautoml\transformers\categorical.py:1029: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cnts = cnts.append(Series([cnts.shape[0] + 1], index=[np.nan]))
c:\Users\Dmitry.Khodykin\Documents\dev\envs\lamlv

In [51]:
test_pred = automl.predict(test_df)
y_predicted = (test_pred.data[:, 0] > 0.5)*1

In [52]:
test_df["label"] = y_predicted

In [53]:
submission = test_df[["id", "label"]]
submission.to_csv("./Out/sample_solution.csv", index=False)

In [54]:
s = pd.read_csv("./Out/sample_solution.csv")

In [55]:
s

,id,label
0,7815282,0
1,8443555,0
2,6352559,0
3,4921420,0
4,1420440,0
...,...,...
3999995,9030800,0
3999996,6275048,0
3999997,4701757,0
3999998,611153,0


In [56]:
s["label"].value_counts()

0    3899335
1     100665
Name: label, dtype: int64